To do:
see https://www.notion.so/aiea/Tasks-8245554f700248038fe006a4c0475ab9?p=125ac685e25d4612b0999d1a32b29de2&pm=s 

In [12]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

In [13]:
SCORES = {
    chess.KING : 200,
    chess.QUEEN : 9,
    chess.ROOK : 5,
    chess.BISHOP : 3,
    chess.KNIGHT : 3,
    chess.PAWN : 1,
}
# Scoring function based on the combined weighted score of multiple smaller scoring functions
def evaluation(b: chess.Board, player: bool):
    material_score = eval_material(b, player)
    return material_score

# Very basic eval function based on counting pieces on the board
def eval_material(b:chess.Board, player: bool):
    if b.is_checkmate():
        return -inf if b.turn == chess.WHITE else inf
    
    # The list needs to be size 7 because PIECE_TYPES goes from 1 to 6
    # Creates a dictionary of all the pieces on the board, and counts them
    piece_count = { c : [0] * 7 for c in chess.COLORS }
    for piece in b.piece_map().values():
        piece_count[piece.color][piece.piece_type] += 1

    # Sums the difference in count of each piece type, multiplied by that type's material value
    total = 0
    for piece in chess.PIECE_TYPES:
        diff = piece_count[chess.WHITE][piece] - piece_count[chess.BLACK][piece]
        total += SCORES[piece] * diff
    
    return total if player else -total

# Test the negation of the piece value based on player colour
# board = chess.Board("K7/8/8/8/8/6Q1/5p2/k3p3 b KQkq - 0 4")
# display(board)
# print(eval_material(board, False))

In [14]:
def minimax(b: chess.Board, player: bool, depth: int):
    return get_max_move(b, player, depth, -inf, +inf)

def get_max_move(b: chess.Board, player: bool, depth: int, alpha: int, beta: int):
    e = evaluation(b, player)
    if depth == 0 or abs(e) == inf:
        return e, None

    best_value = -inf
    best_moves = []
    for m in b.legal_moves:
        temp_board = b.copy()
        temp_board.push(m)
        new_value = get_min_move(temp_board, depth - 1, player, alpha, beta)[0]

        # Maintain a running max of the best move
        if new_value > best_value:
            best_value = new_value
            best_moves = [m]
        elif new_value == best_value:
            best_moves.append(m)

        # Alpha-beta pruning
        if best_value >= beta:
            break
        alpha = max(alpha, best_value)

    return best_value, random.choice(best_moves)

def get_min_move(b: chess.Board, player: bool, depth: int, alpha: int, beta: int):
    e = evaluation(b, player)
    if depth == 0 or abs(e) == inf:
        return e, None
    
    worst_value = +inf
    for m in b.legal_moves:
        temp_board = b.copy()
        temp_board.push(m)
        new_value = get_max_move(temp_board, depth - 1, player, alpha, beta)[0]

        # Maintain a running minimum
        if new_value < worst_value:
            worst_value = new_value

        # Alpha-beta pruning        
        if worst_value <= alpha:
            break
        beta = min(beta, worst_value)

    return worst_value, None


In [15]:
rand = random.Random(0)
def get_random_move(b:chess.Board, *_):
    return "rand :)", rand.choice(list(b.legal_moves))

In [ ]:
MAX_DEPTH = 4
def test_game(p1, p2):
    # board = chess.Board("K7/8/8/8/8/6Q1/5p2/k3p3 b KQkq - 0 4")
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate():
        score, move = current_player(board, MAX_DEPTH, board.turn)
        board.push(move)
        
        
        clear_output(True)
        print("=== Commentary of the below board ===")
        print(f"move chosen = {move}")
        print(f"move score = {score}")
        display(board)
        
        current_player = p1 if board.turn == chess.WHITE else p2
        if input() == "q":
            break

test_game(minimax, get_random_move)

In [6]:
def get_expectimax_move(board:chess.Board, player):
    opponent = not player

    def max_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        v = -inf
        move = None
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            s_val = chance_value(temp, depth + 1)
            if s_val > v:
                v, move = s_val, m
        return v, move

    def chance_value(b:chess.Board, depth:int):
        e = evaluation(b)
        if depth == MAX_DEPTH or abs(e) == inf:
            return e, None
        
        successors = []
        for m in b.legal_moves:
            temp = b.copy()
            temp.push(m)
            successors.append(max_value(temp, depth + 1)[0])
        return sum(successors) / len(successors)
    
    return max_value(board, 0)[1]

In [7]:
def play_game(p1, p2):
    """
    Play a game of chess

    Parameters p1 and p2 represent the white and black players respectively

    They should be one of:

    get_alpha_beta_move,
    get_expectimax_move,
    get_random_move

    Turns must be progressed manually by pressing Enter when prompted
    
    Enter "q" to stop the game
    """
    board = chess.Board()
    display(board)
    current_player = p1

    while not board.is_checkmate():
        move = current_player(board, board.turn)
        board.push(move)
        clear_output(True)
        display(board)

        print("=== Commentary ===")
        print("dummy text")
        
        current_player = p1 if board.turn == chess.WHITE else p2
        if input() == "q":
            break

Below here we did some testing and exploration of the minimax algorithm in a simpler context (a tree)

In [7]:
class Node:
    """
    This class describes a single node contained within a graph.
    It has the following instannce level attributes:

    ID: An integer id for the node i.e. 1
    """
    def __init__(self, id, cost):
        self.id = id
        self.cost = cost
        self.connected_nodes = []

In [4]:
def get_alpha_beta_move_tree(root:Node, player:bool):
    opponent = not player

    def max_value(node:Node, depth):
        if depth == MAX_DEPTH:
            return node.cost, None
        
        v = -inf
        move = None
        for m in node.connected_nodes:
            s_val = min_value(m, depth + 1)
            if s_val > v:
                v, move = s_val, m
        return v, move

    def min_value(node:Node, depth):
        if depth == MAX_DEPTH:
            return node.cost
        
        v = inf
        for m in node.connected_nodes:
            s_val = max_value(m, depth + 1)[0]
            if s_val < v:
                v = s_val
        return v

    return max_value(root, 0)[1]

In [ ]:
a = Node('a', 0)
b = Node('b', 0)
c = Node('c', 0)
d = Node('d', 0)
e = Node('e', 0)
f = Node('f', 0)
g = Node('g', 0)
h = Node('h', 2)
i = Node('i', 3)
j = Node('j', 6)
k = Node('k', 1)
l = Node('l', 9)
m = Node('m', 2)
n = Node('n', 8)
o = Node('o', 5)

a.connected_nodes = [b, c]
b.connected_nodes = [d, e]
c.connected_nodes = [f, g]
d.connected_nodes = [h, i]
e.connected_nodes = [j, k]
f.connected_nodes = [l, m]
g.connected_nodes = [n, o]

MAX_DEPTH = 1
inf = 10000000000000000

print(get_alpha_beta_move_tree(g, True).id)